In [168]:
import pandas as pd
import random
import numpy as np
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import save_model
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
import re
import keras.backend as K
from sklearn.metrics import classification_report

In [169]:
pip install transformers

In [170]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [171]:
validation_test = pd.read_csv('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/validation_test.csv')

df = pd.read_csv('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/training/dataFixBangetTrainingAffan.csv')

In [172]:
import sys
sys.path.append('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/notebooks/modules')

from preprocessing import DataPreProcessing

X, y, one_hot_mapping = DataPreProcessing.PreProcessBatchValidationLSTM(df)

DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Below zero detected in labels data
DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Non-Category datatype detected, converting to Category datatype
DataPreProcessing[PreProcessBatchValidation]: labels shape (13496, 2), input shape (13496,)


In [173]:
df['labels'].value_counts()

-1    6748
 1    6748
Name: labels, dtype: int64

In [174]:
x_train,x_test,y_train,y_test = train_test_split(X,y,
                                                 test_size=0.2, 
                                                 random_state=10)

In [175]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10796,)
(10796, 2)
(2700,)
(2700, 2)


### vectorizer

In [176]:
# Setup text vectorization variables
max_vocab_length = 1000 # max number of words to have in our vocabulary
max_length = 128
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

Beautiful!

To map our `TextVectorization` instance `text_vectorizer` to our data, we can call the `adapt()` method on it whilst passing it our training text.

In [177]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(x_train)

contoh

In [178]:
# Create sample sentence and tokenize it
sample_sentence = "fendi kae affan gas ayo"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 128), dtype=int64, numpy=
array([[  1,   1,   1,   1, 720,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [179]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 1000
Top 5 most common words: ['', '[UNK]', 'nya', 'yang', 'dan']
Bottom 5 least common words: ['tiket', 'tenderloin', 'sukses', 'sempurna', 'seharusnya']


### Creating an Embedding using an Embedding Layer

> Indented block



We've got a way to map our text to numbers. How about we go a step further and turn those numbers into an embedding?

The powerful thing about an embedding is it can be learned during training. This means rather than just being static (e.g. `1` = I, `2` = love, `3` = TensorFlow), a word's numeric representation can be improved as a model goes through data samples.

We can see what an embedding of a word looks like by using the [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer. 

The main parameters we're concerned about here are:
* `input_dim` - The size of the vocabulary (e.g. `len(text_vectorizer.get_vocabulary()`).
* `output_dim` - The size of the output embedding vector, for example, a value of `100` outputs a  feature vector of size 100 for each word.
* `embeddings_initializer` - How to initialize the embeddings matrix, default is `"uniform"` which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.
* `input_length` - Length of sequences being passed to embedding layer.

Knowing these, let's make an embedding layer.

In [180]:
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length) # how long is each input

embedding

contoh

In [181]:
# Get a random sentence from training set
random_sentence = random.choice(x_train)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
aq berdo'a semoga kamu ga lihat pelangi emg kenapa say biar kamu ga nemuin jalan pulang ke kayangan       

Embedded version:


<tf.Tensor: shape=(1, 128, 128), dtype=float32, numpy=
array([[[-0.00155165, -0.00794391, -0.04647359, ..., -0.02207973,
         -0.03493924, -0.04452537],
        [-0.00155165, -0.00794391, -0.04647359, ..., -0.02207973,
         -0.03493924, -0.04452537],
        [ 0.0419164 , -0.00414918, -0.02947918, ...,  0.03877702,
          0.03274295, -0.00446784],
        ...,
        [ 0.01578739,  0.0097748 ,  0.04733063, ...,  0.00038437,
          0.04646469, -0.00925455],
        [ 0.01578739,  0.0097748 ,  0.04733063, ...,  0.00038437,
          0.04646469, -0.00925455],
        [ 0.01578739,  0.0097748 ,  0.04733063, ...,  0.00038437,
          0.04646469, -0.00925455]]], dtype=float32)>

# Model

In [182]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(x_train)
X = tokenizer.texts_to_sequences(x_train)
X = pad_sequences(X)

In [183]:
# MODEL CNN-LSTM
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(x) # return vector for whole sequence
out = tf.keras.layers.Dense(128, activation='relu')(x)
out = tf.keras.layers.Dropout(0.5)(out)
outputs = layers.Dense(2, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")

In [184]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
METRICS = [
        tf.keras.metrics.TruePositives(name='TruePositive'),
        tf.keras.metrics.FalsePositives(name='FalsePositive'),
        tf.keras.metrics.TrueNegatives(name='TrueNegative'),
        tf.keras.metrics.FalseNegatives(name='FalseNegative'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        get_f1
      ]
# Compile model
model.compile(loss="categorical_crossentropy",
                optimizer='adam',
                metrics=METRICS)

In [185]:
model.summary()

Model: "model_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_4 (TextVe (None, 128)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 128, 128)          128000    
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 128, 32)           8224      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 64, 32)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               82432     
_________________________________________________________________
dense_25 (Dense)             (None, 2)                 2

In [186]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-53924150-af0f-ce7f-7c06-db9d76ac8bd4)


In [187]:
epoch = 5
# Fit model

model_history = model.fit(x_train,
                          y_train,
                          epochs=epoch,
                          batch_size=64,
                          validation_data=(x_test, y_test),
                          verbose=1,
                          )

Epoch 1/5
169/169 [==============================] - 51s 277ms/step - loss: 0.5943 - TruePositive: 7305.0000 - FalsePositive: 3824.0000 - TrueNegative: 6972.0000 - FalseNegative: 3491.0000 - accuracy: 0.6643 - precision: 0.6564 - recall: 0.6766 - auc: 0.7376 - get_f1: 0.6655 - val_loss: 0.5302 - val_TruePositive: 2013.0000 - val_FalsePositive: 691.0000 - val_TrueNegative: 2009.0000 - val_FalseNegative: 687.0000 - val_accuracy: 0.7448 - val_precision: 0.7445 - val_recall: 0.7456 - val_auc: 0.8314 - val_get_f1: 0.7435
Epoch 2/5
169/169 [==============================] - 46s 271ms/step - loss: 0.4429 - TruePositive: 8697.0000 - FalsePositive: 2135.0000 - TrueNegative: 8661.0000 - FalseNegative: 2099.0000 - accuracy: 0.8041 - precision: 0.8029 - recall: 0.8056 - auc: 0.8756 - get_f1: 0.8042 - val_loss: 0.4321 - val_TruePositive: 2230.0000 - val_FalsePositive: 470.0000 - val_TrueNegative: 2230.0000 - val_FalseNegative: 470.0000 - val_accuracy: 0.8259 - val_precision: 0.8259 - val_recall: 0.

In [188]:
model_history.history["epochs"] = list(range(1,epoch+1))

In [189]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y= model_history.history["accuracy"],x= model_history.history["epochs"],mode="lines+markers",name="Training accuracy"))
fig.add_trace(go.Scatter(y= model_history.history["loss"],x= model_history.history["epochs"],mode="lines+markers",name="Training loss"))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y= model_history.history["val_accuracy"],x= model_history.history["epochs"],mode="lines+markers",name="Val accuracy"))
fig.add_trace(go.Scatter(y= model_history.history["val_loss"],x= model_history.history["epochs"],mode="lines+markers",name="Val loss"))
fig.show()

In [190]:
# model.save('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/models/LSTMv1/5epoch')

In [191]:
validation_test['labels'].value_counts()

 1    40
-1    40
Name: labels, dtype: int64

In [192]:
# X = np.array(validation_test['tweet'].to_list())
# y = oneHotEncode(np.array(validation_test['labels'].tolist()))
X, y, one_hot_mapping = DataPreProcessing.PreProcessBatchValidationLSTM(validation_test)

DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Below zero detected in labels data
DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Non-Category datatype detected, converting to Category datatype
DataPreProcessing[PreProcessBatchValidation]: labels shape (80, 2), input shape (80,)


In [193]:
model.evaluate(X, y, batch_size=64)

2/2 [==============================] - 0s 25ms/step - loss: 0.7292 - TruePositive: 56.0000 - FalsePositive: 24.0000 - TrueNegative: 56.0000 - FalseNegative: 24.0000 - accuracy: 0.7000 - precision: 0.7000 - recall: 0.7000 - auc: 0.7496 - get_f1: 0.8125


[0.7291857004165649,
 56.0,
 24.0,
 56.0,
 24.0,
 0.699999988079071,
 0.699999988079071,
 0.699999988079071,
 0.7496094107627869,
 0.8125]

In [197]:
model.save('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/models/lstmFIX')

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/models/lstmFIX/assets


In [195]:
def predictText(text):
  return className[np.argmax(model.predict([text]))]

In [196]:
predictText(['saya bahagia wkwkw','anjing kalian semua'])

NameError: ignored